## Analyzing Chinook Database using SQL and Python

The Chinook database contains information about a fictional digital music shop.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

The Chinook database is provided as a SQLite database file called chinook.db.

### the database schema

![schema](https://s3.amazonaws.com/dq-content/280/chinook-schema.svg)

In [1]:
import sqlite3
import pandas as pd


In [2]:
db = 'chinook.db'
conn = sqlite3.connect(db)


### checking some tables

In [3]:

inv = pd.read_sql('select * from invoice', conn)
inv.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [4]:
inv_line = pd.read_sql('select * from invoiceline', conn)
inv_line.head()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


### What are the three most preferred genres?

In [5]:
q1='''SELECT g.name, SUM(quantity) AS quantity_bought
      FROM invoiceline i
      JOIN track t on i.TrackId = t.TrackId
      JOIN genre g on g.GenreId = t.GenreId
      GROUP BY g.name
      ORDER BY SUM(quantity) DESC
      LIMIT 3'''

df1 = pd.read_sql(q1, conn)
df1

,Name,quantity_bought
0,Rock,835
1,Latin,386
2,Metal,264


In [6]:
# Rock takes the lead by a huge margin(more than 50%) followed by Latin.
#It is an important insight in terms of sales and marketing efforts, the company can put more efforts and budget into the most popular genres to boost the sales.

### Which country was the best buyer in 2009?

In [7]:
#in terms of amount
q2 = '''SELECT c.country, SUM(total) AS total
        FROM customer c
        JOIN invoice i on i.CustomerId = c.CustomerId
        WHERE i.InvoiceDate BETWEEN '2009-01-01' AND '2009-12-31'
        GROUP BY country
        ORDER BY SUM(total) DESC
        LIMIT 3'''
df2 = pd.read_sql(q2, conn)
df2

,Country,total
0,USA,103.95
1,Canada,57.42
2,Germany,53.46


In [8]:
#in terms of quantity
q3 = '''SELECT c.country, SUM(quantity) AS total
        FROM customer c
        JOIN invoice i ON i.CustomerId = c.CustomerId
        JOIN invoiceline l ON l.InvoiceId = i.InvoiceId
        WHERE i.InvoiceDate BETWEEN '2009-01-01' AND '2009-12-31'
        GROUP BY country
        ORDER BY SUM(total) DESC
        LIMIT 3'''
df3 = pd.read_sql(q3, conn)
df3

,Country,total
0,USA,105
1,Canada,58
2,Germany,54


In [9]:
# Both in terms of quantity and total cash, the USA leads as the best buyer

### Which was the most popular album ever sold on the playlist?

In [10]:
q4 = '''SELECT a.title, SUM(quantity) as albums_sold
        FROM album a
        JOIN track t ON t.AlbumId = a.AlbumId
        JOIN invoiceline i on i.TrackId = t.TrackId
        GROUP BY title
        ORDER BY SUM(quantity) DESC
        LIMIT 1'''

df4 = pd.read_sql(q4, conn)
df4

,Title,albums_sold
0,Minha Historia,27


In [11]:
# Minha historia was the most popular album

### Who was the customer who spent the most and their favorite genre?

In [12]:
q5 ='''  
        SELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) as total_spent
        FROM customer c
        JOIN invoice i ON i.CustomerId = c.CustomerId
        JOIN invoiceline l ON l.InvoiceId = i.InvoiceId
        JOIN track t on t.TrackId = l.TrackId
        JOIN genre g ON g.GenreId = t.GenreId
        GROUP BY c.CustomerId
        ORDER BY SUM(i.total) DESC
        LIMIT 1
        '''
d5 = pd.read_sql(q5, conn)
d5

,CustomerId,FirstName,LastName,total_spent
0,6,Helena,Holý,502.62


In [13]:
#Hellen of customer id 6 spent the most

In [14]:
q6 =''' SELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) as total ,g.Name
        FROM customer c
        JOIN invoice i ON i.CustomerId = c.CustomerId
        JOIN invoiceline l ON l.InvoiceId = i.InvoiceId
        JOIN track t on t.TrackId = l.TrackId
        JOIN genre g ON g.GenreId = t.GenreId
        GROUP BY c.CustomerId, g.Name
        HAVING c.CustomerId=6
        ORDER BY SUM(i.Total) DESC
               
        '''
d6 = pd.read_sql(q6, conn)
d6

,CustomerId,FirstName,LastName,total,Name
0,6,Helena,Holý,155.16,TV Shows
1,6,Helena,Holý,129.30,Drama
2,6,Helena,Holý,72.39,Rock
3,6,Helena,Holý,47.64,Alternative & Punk
4,6,Helena,Holý,30.69,Latin
5,6,Helena,Holý,25.86,Science Fiction
6,6,Helena,Holý,17.82,Electronica/Dance
7,6,Helena,Holý,17.82,R&B/Soul
8,6,Helena,Holý,5.94,Blues


In [15]:
#Her favourite genre is TV Shows

### which album to advertise in USA i.e which sell best in the USA

In [16]:
q7 ='''WITH usa_tracks AS
        (
         SELECT l.Quantity, l.TrackId,l.InvoiceLineId FROM invoiceline l
         INNER JOIN invoice i ON l.InvoiceId = i.InvoiceId
         INNER JOIN customer c ON i.CustomerId = c.CustomerId
         WHERE c.country = "USA" 
        )
    SELECT 
    g.Name Genre, 
    SUM(ut.Quantity) "Tracks Sold", 
    CAST(SUM(ut.Quantity) AS FLOAT)/(SELECT SUM(Quantity) FROM usa_tracks) "% Tracks Sold"
    FROM usa_tracks ut
    INNER JOIN track t ON ut.TrackId = t.TrackId
    INNER JOIN genre g ON t.GenreId = g.GenreId
    GROUP BY 1 
    ORDER BY 2 DESC
    LIMIT 10'''
d7 = pd.read_sql(q7, conn)
d7

,Genre,Tracks Sold,% Tracks Sold
0,Rock,157,0.317814
1,Latin,91,0.184211
2,Metal,64,0.129555
3,Alternative & Punk,50,0.101215
4,Jazz,22,0.044534
5,Blues,15,0.030364
6,TV Shows,14,0.028340
7,R&B/Soul,12,0.024291
8,Comedy,8,0.016194
9,Classical,8,0.016194


### employee performance

In [17]:
q8 = '''SELECT e.EmployeeId, e.FirstName, e.HireDate,SUM(i.Total) as amount_sold
        FROM  employee e
        JOIN customer c ON c.SupportRepId = e.EmployeeId
        JOIN invoice i ON i.CustomerId = c.CustomerId
        GROUP BY e.EmployeeId
        ORDER BY SUM(i.Total) DESC'''
d8 = pd.read_sql(q8, conn)
d8

,EmployeeId,FirstName,HireDate,amount_sold
0,3,Jane,2002-04-01 00:00:00,833.04
1,4,Margaret,2003-05-03 00:00:00,775.40
2,5,Steve,2003-10-17 00:00:00,720.16


### Customers by Country

In [18]:
q9 = '''SELECT c.Country, COUNT(c.CustomerId) total_customers, SUM(i.Total) total_sales,
        CAST(SUM(i.Total) AS FLOAT)/SUM(l.Quantity) avg_order_value
        FROM customer c
        JOIN invoice i on i.CustomerId = c.CustomerId
        JOIN invoiceline l on l.InvoiceId = i.InvoiceId
        GROUP BY 1
        ORDER BY 2 DESC
        '''
d9 = pd.read_sql(q9, conn)
d9

,Country,total_customers,total_sales,avg_order_value
0,USA,494,4667.06,9.447490
1,Canada,304,2689.96,8.848553
2,France,190,1722.10,9.063684
3,Brazil,190,1677.10,8.826842
4,Germany,152,1392.48,9.161053
5,United Kingdom,114,1003.86,8.805789
6,Portugal,76,687.24,9.042632
7,Czech Republic,76,879.24,11.568947
8,India,74,667.28,9.017297
9,Sweden,38,340.62,8.963684
